In [1]:
import torch
import sys
sys.path.append("/home/msst/repo/Quantization")
import qlib
DEVICE = 'cuda:0'

In [2]:
m = 1024
n = 2048

w = torch.randn(m, n, device=DEVICE)

quantizer = qlib.trellis_quantizer(
	L=10,
	K=3, 
	V=2, 
	T=256,
	decode_mode="LowBitSym", 
	tlut_bits=10
).to(DEVICE)

reco, states = quantizer.quantize(w)
packed = quantizer.pack_trellis(states)
reco_ref = quantizer.reconstruct_weight(packed, w.shape)

err_ref  = torch.mean((reco_ref  - w)**2, dim=-1)
print(f"error (unpack): {err_ref.mean():.3f} ± {err_ref.std():.3f}")


reco_fast = quantizer.reconstruct_weight_fast(packed, w.shape)

err_fast = torch.mean((reco_fast - w)**2, dim=-1)
print(f"error (fast)  : {err_fast.mean():.3f} ± {err_fast.std():.3f}")


100%|██████████| 32/32 [00:00<00:00, 105.67it/s]


error (unpack): 0.029 ± 0.002


AssertionError: 

In [3]:
%%timeit
reco = quantizer.reconstruct_weight(packed, w.shape)

5.13 ms ± 62.1 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
%%timeit
reco_fast = quantizer.reconstruct_weight_fast(packed, w.shape)

106 μs ± 74.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
